#### Download libraries if needed

In [11]:
%pip install numpy
%pip install pdfminer.six

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Import libraries (NumPy + Re + one library to read the .pdf file,)

In [12]:
import re
import numpy as np
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer

#### Read the text from pdf using pdfminer

In [ ]:

def read_pdf(pdf_path):
    text_per_page = {}
    for pagenum, page in enumerate(extract_pages(pdf_path)):
        page_content = []
        page_elements = [element for element in page._objs]
        for element in page_elements:
            if isinstance(element, LTTextContainer):
                line_text = element.get_text()
                page_content.append(line_text)  
        text_per_page[pagenum] = ''.join(page_content)
    return ''.join(text_per_page.values())

pdf_path1 = '../vector_word.pdf'
pdf_path2 = '../transformers.pdf'
text1 = read_pdf(pdf_path1)
text2 = read_pdf(pdf_path2)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

#### Text preprocessing
- make stop word list
- remove special characters and tags from the documents
- split text on word using regular expression

In [14]:
stop_words = {"","s","d","y", "v", "j", "e","g", "u", "t", "n", "c", "o", "l", "i", "a", "x", "b", "r", "cs", "ba", "aapo", "able", "zi", "ym", "of", "edu", "th", "it", "at", "or", "its", "you", "did", "pro", "such", "about", "thus", "or", "gmail", "with",  "arxiv", "our", "yee", "other", "from", "ing", "uses", "while", "but", "and", "ieee", "due","els", "else", "their", "inc", "com", "the", "is", "an", "for", "that", "in", "we", "this", "by", "also", "to", "be", ",", "et", "al", "figure", "on", "are", "as", "than","kai", "aaai", "tion"}

def text_preprocessing(text):
    words = re.split(r"[\s*\s\.\]\[\:\@\-]" , text.lower())
    for i in range(len(words)):
        words[i] = re.sub(r"[^a-z]",'', words[i])
    words = [word for word in words if word not in stop_words]
    return words

clean_text1 = text_preprocessing(text1)
clean_text2 = text_preprocessing(text2)
clean_text1

['distributed',
 'representations',
 'words',
 'phrases',
 'compositionality',
 'tomas',
 'mikolov',
 'google',
 'mountain',
 'view',
 'mikolov',
 'google',
 'ilya',
 'sutskever',
 'google',
 'mountain',
 'view',
 'ilyasu',
 'google',
 'chen',
 'google',
 'mountain',
 'view',
 'google',
 'greg',
 'corrado',
 'google',
 'mountain',
 'view',
 'gcorrado',
 'google',
 'jeffrey',
 'dean',
 'google',
 'mountain',
 'view',
 'jeff',
 'google',
 'abstract',
 'recently',
 'introduced',
 'continuous',
 'skip',
 'gram',
 'model',
 'efcient',
 'method',
 'learning',
 'high',
 'quality',
 'distributed',
 'vector',
 'representations',
 'capture',
 'large',
 'num',
 'ber',
 'precise',
 'syntactic',
 'semantic',
 'word',
 'relationships',
 'paper',
 'present',
 'several',
 'extensions',
 'improve',
 'both',
 'quality',
 'vectors',
 'training',
 'speed',
 'subsampling',
 'frequent',
 'words',
 'obtain',
 'signicant',
 'speedup',
 'learn',
 'more',
 'regular',
 'word',
 'representations',
 'describe',
 '

#### Create bag of words model
### Steps:
- combine corpus in one array
- make vocabulary
- delete word that appearce only 1 times in both text to shrink bow mode(it will delete names and arrors that we do not need in our vector)
- count word from vocabulary appearance in texts from corpus
- return vector of counts 

In [15]:
def bag_of_word(corpus):
    full_texts =  [text for each_text in corpus for text in each_text]
    min_appereance = 2
    texts_dict = sorted(set(full_texts))
    clean_dict = [word for word in texts_dict if full_texts.count(word) >= min_appereance]
    vector = []
    vector = np.zeros((len(corpus), len(clean_dict)), dtype=np.uint32)
    for i, text in enumerate(corpus):
        for j, word in enumerate(clean_dict):
            vector[i][j] = text.count(word)
    return vector, clean_dict

bow, texts_dict = bag_of_word([clean_text1, clean_text2])
bow

array([[1, 2, 0, ..., 0, 0, 0],
       [1, 1, 6, ..., 2, 4, 2]], dtype=uint32)

##### Calculate cosine similarity between articles
- interpret measured cosine similarity

In [16]:
def cosine_sim(a, b):
    return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b))

cosine_similarity = cosine_sim(bow[0], bow[1])
print(f"{cosine_similarity * 100:.2f} % similarity")

34.28 % similarity
